In [ ]:
!kaggle datasets download -d suraj520/imdb-tv-series-data
!unzip *.zip

Dataset URL: https://www.kaggle.com/datasets/suraj520/imdb-tv-series-data
License(s): CC0-1.0
 94% 25.0M/26.5M [00:02<00:00, 12.4MB/s]
100% 26.5M/26.5M [00:02<00:00, 11.4MB/s]
Archive:  imdb-tv-series-data.zip
  inflating: action_series.csv       
  inflating: adventure_series.csv    
  inflating: animation_series.csv    
  inflating: biography_series.csv    
  inflating: comedy_series.csv       
  inflating: crime_series.csv        
  inflating: documentary_series.csv  
  inflating: drama_series.csv        
  inflating: family_series.csv       
  inflating: fantasy_series.csv      
  inflating: history_series.csv      
  inflating: horror_series.csv       
  inflating: music_series.csv        
  inflating: musical_series.csv      
  inflating: mystery_series.csv      
  inflating: romance_series.csv      
  inflating: sci-fi_series.csv       
  inflating: sport_series.csv        
  inflating: superhero_series.csv    
  inflating: thriller_series.csv     
  inflating: war_series.csv   

In [ ]:
import pandas as pd
import glob
import re
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
files = glob.glob('*.csv')
files

['crime_series.csv',
 'animation_series.csv',
 'adventure_series.csv',
 'drama_series.csv',
 'fantasy_series.csv',
 'action_series.csv',
 'comedy_series.csv',
 'superhero_series.csv',
 'war_series.csv',
 'romance_series.csv',
 'history_series.csv',
 'thriller_series.csv',
 'documentary_series.csv',
 'family_series.csv',
 'horror_series.csv',
 'sport_series.csv',
 'music_series.csv',
 'biography_series.csv',
 'musical_series.csv',
 'western_series.csv',
 'sci-fi_series.csv',
 'mystery_series.csv']

In [ ]:
df = pd.read_csv('war_series.csv')

In [ ]:
df.sample(5)

,Title,IMDb ID,Release Year,Genre,Cast,Synopsis,Rating,Runtime,Certificate,Number of Votes,Gross Revenue
578,The Auschwitz Report,tt9415108,2021,"Drama, History, War","Director:, Peter Bebjak, | , Stars:, Noel ...",This is the true story of Freddy and Walter--t...,6.6,94 min,NaN,2441,NaN
944,1864,tt1753353,2014,"Drama, History, War","Stars:, Pilou Asbæk, , Sarah-Sofie Boussnina, ...",When Prussia and Austria declare war on Denmar...,6.8,460 min,TV-14,3914,NaN
7075,The Diary of Anne Frank,tt0923648,1962 TV Movie,"Drama, War","Stars:, Kitty Courbois, , Martine Crefcour, , ...","During World War II, a teenage Jewish girl nam...",6.1,90 min,NaN,13,NaN
10540,Black Hawk Down,tt0265086,2001,"Action, Drama, History","Director:, Ridley Scott, | , Stars:, Josh ...",The story of 160 elite U.S. soldiers who dropp...,7.7,144 min,R,407639,"108,638,745"
7706,Pariahs of Glory,tt0058452,1964,"Drama, War","Director:, Henri Decoin, | , Stars:, Curd ...",Ferrier is fighting in Indochina ; a German pi...,5.7,100 min,NaN,44,NaN


In [ ]:
dfs = pd.DataFrame()
for file in files:
  df = pd.read_csv(file)
  dfs = pd.concat([dfs, df], axis='rows')

<ipython-input-6-afb719318ad2>:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfs = pd.concat([dfs, df], axis='rows')


In [ ]:
dfs.shape

(236828, 11)

In [ ]:
dfs.sample(5)

,Title,IMDb ID,Release Year,Genre,Cast,Synopsis,Rating,Runtime,Certificate,Number of Votes,Gross Revenue
1687,The Adventures of Super Mario Bros. 3,tt0098762,1990,"Animation, Adventure, Comedy","Stars:, Walker Boone, , Tony Rosato, , Tracey ...",Join Mario and Luigi in 26 action-packed adven...,7.1,11 min,TV-Y7,1837,NaN
6042,Vallatud kurvid,tt0142954,1959,"Comedy, Sport","Directors:, Kaljo Kiisk, , Yuli Kun, | , S...",Vaike arrives at the racetrack where the bikes...,7.3,89 min,NaN,243,NaN
3432,The Gnome-Mobile,tt0061715,1967,"Adventure, Family, Fantasy","Director:, Robert Stevenson, | , Stars:, W...","In the Redwood Forests of California, a multi-...",6.4,84 min,Approved,2327,NaN
9589,Panjaa,tt2091384,2011,"Action, Crime, Drama","Director:, Vishnuvardhan, | , Stars:, Pawa...",Ordered by his boss to take out a former gang ...,6.5,156 min,Not Rated,4052,NaN
6839,Totally Spies! The Movie,tt1042916,2009,"Animation, Action, Adventure","Directors:, Pascal Jardin, , Seok-hoon Lee, | ...","In their first mission as, like, total spies, ...",6.3,75 min,TV-Y7-FV,751,NaN


In [ ]:
for column in dfs.columns:
  nan = dfs[column].isna().sum()
  print(f"{column}: {nan}")

Title: 0
IMDb ID: 0
Release Year: 9
Genre: 0
Cast: 872
Synopsis: 0
Rating: 0
Runtime: 19845
Certificate: 67737
Number of Votes: 0
Gross Revenue: 191217


In [ ]:
dfs.Certificate.unique()

array(['TV-MA', 'R', 'PG-13', 'TV-14', nan, 'TV-PG', 'Not Rated', 'PG',
       'Approved', 'TV-Y7', 'Passed', 'G', '16+', 'TV-G', 'GP', 'M',
       'TV-Y7-FV', 'NC-17', 'M/PG', '18+', 'Unrated', 'TV-Y', 'TV-13',
       'T', 'X', '13+', 'AO', 'E', '12', 'E10+', 'MA-13', 'Open', 'F',
       'K-A', 'MA-17', 'EM', 'GA', 'TV_MA', '18'], dtype=object)

In [ ]:
to_drop = ['IMDb ID', 'Cast', 'Gross Revenue', 'Synopsis', 'Release Year']
dfs.drop(to_drop, axis='columns', inplace=True)

In [ ]:
dfs.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes
10171,Weird: The Al Yankovic Story,"Comedy, Crime, Music",6.9,108 min,TV-14,29655
4675,Good Vibes,"Animation, Comedy",7.4,NaN,TV-14,997
2339,Blue Streak,"Action, Comedy, Crime",6.3,93 min,PG-13,86029
272,The Apple Dumpling Gang,"Comedy, Family, Western",6.3,100 min,G,6210
3187,Above the Rim,"Crime, Drama, Sport",6.6,96 min,R,16417


In [ ]:
for column in dfs.columns:
  nan = dfs[column].isna().sum()
  print(f"{column}: {nan}")

Title: 0
Genre: 0
Rating: 0
Runtime: 19845
Certificate: 67737
Number of Votes: 0


In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236828 entries, 0 to 11847
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Title            236828 non-null  object 
 1   Genre            236828 non-null  object 
 2   Rating           236828 non-null  float64
 3   Runtime          216983 non-null  object 
 4   Certificate      169091 non-null  object 
 5   Number of Votes  236828 non-null  object 
dtypes: float64(1), object(5)
memory usage: 12.6+ MB


In [ ]:
dfs.Runtime = dfs.Runtime.apply(lambda x: str(x).replace(' min', ''))

In [ ]:
dfs.Runtime.value_counts()

,count
Runtime,
nan,19845
60,10272
30,7753
90,7178
100,4230
...,...
"1,352",1
740,1
710,1


In [ ]:
dfs['Runtime'] = pd.to_numeric(dfs['Runtime'], errors='coerce')
dfs.Runtime.fillna(60, inplace=True)

In [ ]:
dfs.Runtime.value_counts()

,count
Runtime,
60.0,30160
30.0,7753
90.0,7178
100.0,4230
95.0,4087
...,...
616.0,1
834.0,1
689.0,1


In [ ]:
dfs.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes
731,Space Jam,"Animation, Adventure, Comedy",6.5,88.0,PG,204465
1386,The Cheap Detective,"Comedy, Crime, Mystery",6.4,92.0,PG,5364
2954,Freckles,"Drama, Western",5.5,83.0,NaN,119
7902,The Luminaries,"Adventure, Drama, Mystery",6.4,60.0,TV-MA,4020
8399,Bells Are Ringing,"Comedy, Musical, Romance",6.9,126.0,Not Rated,3585


In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236828 entries, 0 to 11847
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Title            236828 non-null  object 
 1   Genre            236828 non-null  object 
 2   Rating           236828 non-null  float64
 3   Runtime          236828 non-null  float64
 4   Certificate      169091 non-null  object 
 5   Number of Votes  236828 non-null  object 
dtypes: float64(2), object(4)
memory usage: 12.6+ MB


In [ ]:
dfs.Runtime.value_counts()

,count
Runtime,
60.0,30160
30.0,7753
90.0,7178
100.0,4230
95.0,4087
...,...
616.0,1
834.0,1
689.0,1


In [ ]:
for column in dfs.columns:
  nan = dfs[column].isna().sum()
  print(f"{column}: {nan}")

Title: 0
Genre: 0
Rating: 0
Runtime: 0
Certificate: 67737
Number of Votes: 0


In [ ]:
dfs['Certificate'].fillna('unknown', inplace=True)

In [ ]:
for column in dfs.columns:
  nan = dfs[column].isna().sum()
  print(f"{column}: {nan}")

Title: 0
Genre: 0
Rating: 0
Runtime: 0
Certificate: 0
Number of Votes: 0


In [ ]:
dfs.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes
4782,The Tuba Thieves,"Documentary, Drama",6.6,92.0,unknown,33
3998,BraveStarr,"Animation, Action, Adventure",7.0,30.0,TV-Y7,3077
6023,Halo Legends,"Animation, Action, Adventure",6.5,120.0,PG-13,5738
1445,The Postman Always Rings Twice,"Crime, Drama, Romance",6.6,122.0,R,24738
1931,The Outsider,"Action, Crime, Drama",6.2,120.0,TV-MA,27563


In [ ]:
dfs['Number of Votes'] = pd.to_numeric(dfs['Number of Votes'], errors='coerce')

In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236828 entries, 0 to 11847
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Title            236828 non-null  object 
 1   Genre            236828 non-null  object 
 2   Rating           236828 non-null  float64
 3   Runtime          236828 non-null  float64
 4   Certificate      236828 non-null  object 
 5   Number of Votes  236828 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 12.6+ MB


In [ ]:
dfs['Genre'] = dfs['Genre'].apply(lambda x: x.split(','))

In [ ]:
dfs.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes
2795,Ghostbusters,"[Animation, Adventure, Comedy]",6.3,25.0,TV-Y7,3065
592,I Wanna Hold Your Hand,"[Comedy, Music, Romance]",6.8,104.0,PG,5329
510,The Trials of Gabriel Fernandez,"[Documentary, Crime]",8.1,331.0,TV-MA,12597
6920,Dorbees: Making Decisions,"[Animation, Short, Adventure]",2.0,44.0,Not Rated,153
6963,Mighty Magiswords,"[Animation, Short, Action]",4.6,3.0,TV-Y7,1350


In [ ]:
dfs = dfs.reset_index(drop=True)

In [ ]:
mlb = MultiLabelBinarizer()
binary_genres = mlb.fit_transform(dfs['Genre'])
genres_dfs = pd.DataFrame(binary_genres, columns=mlb.classes_)

In [ ]:
dfs_f = pd.concat([dfs, genres_dfs], axis=1)

In [ ]:
dfs_f.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes,Action,Adult,Adventure,Animation,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
159853,X,"[Horror, Mystery, Thriller]",6.6,105.0,R,131606,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91122,Lone Survivor,"[Action, Biography, Drama]",7.5,121.0,R,295691,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119442,An Affair,"[Drama, Thriller]",5.5,92.0,unknown,2360,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71211,Pride,"[Biography, Comedy, Drama]",7.8,119.0,R,59456,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164375,The Great American Cowboy,"[Documentary, Sport]",6.9,89.0,G,115,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dfs_final = dfs_f

In [ ]:
dfs_final = pd.get_dummies(dfs_f['Certificate'], dtype=int)

In [ ]:
dfs_final.sample(5)

,12,13+,16+,18,18+,AO,Approved,E,E10+,EM,...,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,TV_MA,Unrated,X,unknown
165721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
14631,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
171005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dfs_final = pd.concat([dfs_f, dfs_final], axis='columns')
dfs_final.sample(5)

,Title,Genre,Rating,Runtime,Certificate,Number of Votes,Action,Adult,Adventure,Animation,...,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,TV_MA,Unrated,X,unknown
172208,I'm Still Here,"[Comedy, Drama, Music]",6.2,108.0,R,23297,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
188978,They Shall Not Grow Old,"[Documentary, Biography, History]",7.9,47.0,unknown,68,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86680,Rogue's March,"[Adventure, Drama, War]",6.0,84.0,Passed,162,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36090,None But the Brave,"[Adventure, Drama, War]",6.4,106.0,Approved,2008,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122178,A besúgó,"[Drama, Thriller]",8.1,43.0,unknown,3329,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
to_drop = ['Genre', 'Certificate']
dfs_final.drop(to_drop, axis='columns', inplace=True)

In [ ]:
dfs_final.sample(5)

,Title,Rating,Runtime,Number of Votes,Action,Adult,Adventure,Animation,Biography,Comedy,...,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,TV_MA,Unrated,X,unknown
73899,Air America,5.8,113.0,29632,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
199149,Sunny,5.3,78.0,185,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
161174,Influencer,6.1,92.0,2274,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
104243,New Girl,7.8,22.0,233972,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180782,Ek Lief Jou,2.4,94.0,35,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
dfs_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236828 entries, 0 to 236827
Data columns (total 97 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Title            236828 non-null  object 
 1   Rating           236828 non-null  float64
 2   Runtime          236828 non-null  float64
 3   Number of Votes  236828 non-null  int64  
 4    Action          236828 non-null  int64  
 5    Adult           236828 non-null  int64  
 6    Adventure       236828 non-null  int64  
 7    Animation       236828 non-null  int64  
 8    Biography       236828 non-null  int64  
 9    Comedy          236828 non-null  int64  
 10   Crime           236828 non-null  int64  
 11   Documentary     236828 non-null  int64  
 12   Drama           236828 non-null  int64  
 13   Family          236828 non-null  int64  
 14   Fantasy         236828 non-null  int64  
 15   Film-Noir       236828 non-null  int64  
 16   Game-Show       236828 non-null  int6

In [ ]:
Sscaler = StandardScaler()
to_scale = ['Rating', 'Runtime', 'Number of Votes']
dfs_final[to_scale] = Sscaler.fit_transform(dfs_final[to_scale])

In [ ]:
dfs_final.sample(5)

,Title,Rating,Runtime,Number of Votes,Action,Adult,Adventure,Animation,Biography,Comedy,...,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,TV_MA,Unrated,X,unknown
120123,Sonatine,0.597780,0.132984,-0.202086,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
134955,Moonshiners: American Spirit,1.120816,-0.487391,-0.323019,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
111778,Sons of the Good Earth,-0.149415,0.370187,-0.322563,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
168355,Gekijô-ban Mejâ: Yûjô no uiningu shotto,0.149463,0.242462,-0.322873,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
58779,Star Trek Beyond,0.224182,0.643882,1.046078,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_small = dfs_final.iloc[:10000]
df_small_noTitle = df_small.drop('Title', axis='columns')

In [ ]:
df_small.sample(4)

,Title,Rating,Runtime,Number of Votes,Action,Adult,Adventure,Animation,Biography,Comedy,...,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,TV_MA,Unrated,X,unknown
99,"Murder, She Wrote",0.373621,-0.669854,-0.178664,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7915,Detective Conan: The Bride of Halloween,0.074743,0.443172,-0.316225,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7443,Pretty Poison,0.224182,0.041753,-0.302413,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3935,Conversations with a Killer: The Jeffrey Dahme...,0.448341,1.702169,-0.280110,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
df_small_noTitle.shape

(10000, 96)

In [ ]:
similarity_matrix = cosine_similarity(df_small_noTitle)

In [ ]:
similarity_df = pd.DataFrame(similarity_matrix, index=df_small['Title'], columns=df_small['Title'])

In [ ]:
def suggest_seris(name, sugg_number=3):
  name = name.title()
  series = []
  try:
    similar_movies = similarity_df[name].sort_values(ascending=False)
  except TypeError:
    return list(similarity_df[name][:sugg_number].index)
    series.append(list(similar_movies[:sugg_number].index))
  return series.remve(name)

In [ ]:
print(suggest_seris('Animal Kingdom', 5))

['Barry', 'John Wick: Chapter 4', 'Fast X', 'Love & Death', 'Killers of the Flower Moon']


In [ ]:
similarity_df.to_csv('Series_similarity.csv')